# Splunk App for Data Science and Deep Learning - Notebook Template 

This notebook contains a barebone example workflow how to work on custom containerized code that seamlessly interfaces with you Splunk platform by utilizing the Splunk App for Data Science and Deep Learning (DSDL) - formerly known as the Deep Learning Toolkit for Splunk (DLTK). Find more examples and information in the app and on the [DSDL splunkbase page](https://splunkbase.splunk.com/app/4607/#/details).

Note: By default every time you save this notebook the stages main cells are exported into a python module which can then get invoked by Splunk's MLTK SPL commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the DSDL app for more information about this workflow.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import os
# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

numpy version: 1.26.4
pandas version: 2.2.3


## Stage 1 - get a data sample from Splunk
There are currently 2 ways to retrieve data from Splunk: Option 1 is to interactively pull data from Splunk into the DSDL Jupyter Lab environment. This is useful when the Splunk REST API is accessible from the Jupyter environment and a valid Splunk auth token is defined in the DSDL app. This option has advantages to quickly experiment with different Splunk SPL queries and further interactively work with the search results in Jupyter.

### Option 1 - pull data from Splunk

In [3]:
from dsdlsupport import SplunkSearch as SplunkSearch

In [4]:
search = SplunkSearch.SplunkSearch()

In [5]:
df = search.as_df()
df

,_time,feature_0,feature_1,feature_2,i,s
0,2024-09-20T17:39:24.000+02:00,0.7479193396866322,0.6095915557816625,0.35341535741463304,1,1
1,2024-09-20T17:39:24.000+02:00,0.7684882110916078,0.9494387060403824,0.3567090374417603,2,2
2,2024-09-20T17:39:24.000+02:00,0.4815087844617665,0.5215603411197662,0.5110989357344806,3,0
3,2024-09-20T17:39:24.000+02:00,0.7850361596792936,0.6264563561417162,0.730981633067131,4,1
4,2024-09-20T17:39:24.000+02:00,0.18881695484742522,0.46552519127726555,0.16302273701876402,5,2
5,2024-09-20T17:39:24.000+02:00,0.37600346421822906,0.10779458377510309,0.009006098844110966,6,0
6,2024-09-20T17:39:24.000+02:00,0.36187303718179464,0.4258803175762296,0.23626281367614865,7,1
7,2024-09-20T17:39:24.000+02:00,0.7082957876846194,0.2869397671893239,0.6502283890731633,8,2
8,2024-09-20T17:39:24.000+02:00,0.7963134637102485,0.8004127754829824,0.8560442295856774,9,0
9,2024-09-20T17:39:24.000+02:00,0.8209263710305095,0.24220526218414307,0.6967716538347304,10,1


### Option 2 - push data from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. You utilize the `mode=stage` flag in the in the `| fit` command to do this. The search results are accessible then as csv file with the same model name that is defined in the `into app:<modelname>` part of the fit statement. Additionally, meta data is retrieved and accessible as json file. In the same way you can further work with the meta data object as it is exposed in the fit and apply function definitions below in stage 3 and 4.

| makeresults count=10<br>
| streamstats c as i <br>
| eval s = i%3 <br>
| eval feature_{s}=0 <br>
| foreach feature_* [eval \<\<FIELD\>\>=random()/pow(2,31)]<br>
| fit MLTKContainer mode=stage algo=barebone_template _time feature_* i into app:barebone_template<br>

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [14]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("/srv/notebooks/data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("/srv/notebooks/data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [15]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("barebone_template")

In [16]:
df

,_time,i,feature_1,feature_2,feature_0
0,1725374037,1,0.031864,0.284246,0.766344
1,1725374037,2,0.530662,0.393207,0.390361
2,1725374037,3,0.726139,0.496824,0.115615
3,1725374037,4,0.653525,0.426098,0.837128
4,1725374037,5,0.035622,0.322095,0.554212
5,1725374037,6,0.925372,0.668078,0.979063
6,1725374037,7,0.830713,0.803215,0.445145
7,1725374037,8,0.906020,0.782279,0.312434
8,1725374037,9,0.964099,0.633082,0.340853
9,1725374037,10,0.807695,0.792692,0.248587


In [17]:
param

{'options': {'params': {'mode': 'stage', 'algo': 'barebone_template'},
  'args': ['_time', 'feature*', 'i'],
  'feature_variables': ['_time', 'feature*', 'i'],
  'model_name': 'barebone_template3',
  'algo_name': 'MLTKContainer',
  'mlspl_limits': {'handle_new_cat': 'default',
   'max_distinct_cat_values': '100',
   'max_distinct_cat_values_for_classifiers': '100',
   'max_distinct_cat_values_for_scoring': '100',
   'max_fit_time': '600',
   'max_inputs': '100000',
   'max_memory_usage_mb': '4000',
   'max_model_size_mb': '30',
   'max_score_time': '600',
   'use_sampling': 'true'},
  'kfold_cv': None},
 'feature_variables': ['_time', 'feature_1', 'feature_2', 'feature_0', 'i']}

## Stage 2 - create and initialize a model

In [18]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
    model['hyperparameter'] = 42.0
    return model

In [19]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

{'hyperparameter': 42.0}


## Stage 3 - fit the model

In [20]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    # model.fit()
    info = {"message": "model trained"}
    return info

In [21]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

{'message': 'model trained'}


## Stage 4 - apply the model

In [22]:
# apply your model
# returns the calculated results
def apply(model,df,param):
    y_hat = df.index
    result = pd.DataFrame(y_hat, columns=['index'])
    return result

In [23]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

   index
0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9


## Stage 5 - save the model

In [24]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        json.dump(model, file)
    return model

## Stage 6 - load the model

In [25]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = {}
    with open(MODEL_DIRECTORY + name + ".json", 'r') as file:
        model = json.load(file)
    return model

## Stage 7 - provide a summary of the model

In [26]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

After implementing your fit, apply, save and load you can train your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer algo=barebone s from feature_* into app:barebone_model<br>

Or apply your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| apply barebone_model as the_meaning_of_life

## Send data back to Splunk HEC
When you configured the Splunk HEC Settings in the DSDL app you can easily send back data to an index with [Splunk's HTTP Event Collector (HEC)](https://docs.splunk.com/Documentation/Splunk/latest/Data/UsetheHTTPEventCollector). Read more about data formats and options in the [documentation](https://docs.splunk.com/Documentation/Splunk/latest/Data/FormateventsforHTTPEventCollector#Event_metadata).

### Use cases
- you want to offload longer running, possibly distributed computations that need to deliver results asynchroneously back into Splunk. 
- you might not want to present results back into the search pipeline after your `| fit` or `| apply` command. 
- you can easily utilize this approach for any logging purposes or other profiling tasks in your ML code so you can actively monitor and analyze your processes.

### Example

In [27]:
import os
import sys
import time
import json
import numpy as np
import pandas as pd
import requests
from datetime import datetime, timezone

class SplunkHEC(object):
    def __init__(self, url='', token=''):
        self.url = url + "/services/collector/event"
        self.token = token
        self.authHeader = ''

        print(f"Initial URL from constructor: {self.url}")
        print(f"Initial Token from constructor: {self.token}")

        if "splunk_hec_enabled" in os.environ:
            access_enabled = os.environ["splunk_hec_enabled"]
            if access_enabled == "1":
                self.url = os.environ["splunk_hec_url"] + "/services/collector/event"
                self.token = os.environ["splunk_hec_token"]

                print(f"URL from environment: {self.url}")
                print(f"Token from environment: {self.token}")

        if self.token:
            self.authHeader = {'Authorization': 'Splunk {}'.format(self.token)}

    def send_to_hec(self, jsonDict, url, token):
        return requests.post(url, headers=self.authHeader, json=jsonDict, verify=False)
    
    def send(self, events):
        return self.send_to_hec(events, self.url, self.token)

    def send_hello_world(self, num=1):
        data = []
        for i in range(0, num):
            event = {'event': {'message': 'hello world ' + str(i)}, 'time': datetime.now().timestamp()}
            time.sleep(0.001)
            data.append(event)
        return self.send(data)

    def get_url(self):
        return self.url

In [28]:
# Create an instance of the class
hec = SplunkHEC()

# Print the URL using the get_url method
print(f"URL with environment variables: {hec.get_url()}")

Initial URL from constructor: /services/collector/event
Initial Token from constructor: 
URL from environment: http://host.docker.internal:8088/services/collector/event
Token from environment: cdf776b5-b257-41e5-8002-eaa14fcb9354
URL with environment variables: http://host.docker.internal:8088/services/collector/event


In [29]:
from dsdlsupport import SplunkHEC as SplunkHEC
hec = SplunkHEC.SplunkHEC()

In [30]:
import os

# Check if the environment variable exists
if "splunk_hec_url" in os.environ:
    splunk_hec_url = os.environ["splunk_hec_url"]
    print(f"The value of splunk_hec_url is: {splunk_hec_url}")
else:
    print("The environment variable splunk_hec_url does not exist.")

The value of splunk_hec_url is: http://host.docker.internal:8088


In [31]:
# example to send 10 hello world events
response = hec.send_hello_world(10)

In [32]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

HEC endpoint http://host.docker.internal:8088/services/collector/event 
returned with status code 200 
and response message: {"text":"Success","code":0}


In [33]:
# example to send a JSON object, e.g. to log some data
from datetime import datetime
response = hec.send({'event': {'message': 'operation done', 'log_level': 'INFO' }, 'time': datetime.now().timestamp()})

In [34]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

HEC endpoint http://host.docker.internal:8088/services/collector/event 
returned with status code 200 
and response message: {"text":"Success","code":0}


## End of Stages
All subsequent cells are not tagged and can be used for further freeform code